In [735]:
import pandas as pd
import os

In [736]:
files=[file for dirpath,directory,file in os.walk('./all_csv/')][0]

In [737]:
match_data=pd.DataFrame(data=None)
odi_scorecard=pd.DataFrame(data=None)
ttwenty_scorecard=pd.DataFrame(data=None)
odi_info=pd.DataFrame(data=None)
ttwenty_info=pd.DataFrame(data=None)

In [738]:
def rename_date_umpire(index_list):
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='date':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='umpire':
            index_list[i]+='_'+str(n)
            n+=1
    return index_list

In [739]:
def find_game(df_game,df_info):
    if 'series' in df_info.columns:
        if 'odi' in str.lower(df_info.iloc[0]['series']):
            return 'odi'
        if 't20i' in str.lower(df_info.iloc[0]['series']):
            return 'twenty'
    if max(df_game['balls_bowl'])<=24:
        return 'twenty'
    if 24<max(df_game['balls_bowl'])<=60:
        return 'odi'
    return

In [740]:
def append_file(temp_df,temp_info_df,type_game):
    global odi_scorecard
    global ttwenty_scorecard
    global odi_info
    global ttwenty_info
    if type_game=='odi':
        odi_scorecard=odi_scorecard.append(temp_df,ignore_index=True)
        odi_info=odi_info.append(temp_info_df,ignore_index=True)
    elif type_game=='twenty':
        ttwenty_scorecard=ttwenty_scorecard.append(temp_df,ignore_index=True)
        ttwenty_info=ttwenty_info.append(temp_info_df,ignore_index=True)

In [741]:
def get_extras_type(match_data):
    list_extras=[]
    for index,row in match_data.iterrows():
        ov=str(row['over']).split('.')
        ball_no=int(ov[1])
        over_no=int(ov[0])
        if row['extras']!=0:
            if (row['extras']==1 or row['extras']==3) and index+1<match_data.shape[0] and (match_data.loc[index+1,'striker']==row['non-striker'] or match_data.loc[index+1,'non-striker']==row['striker']):
                match_data.loc[index,'extras_type']='b'
            else:
                list_extras.append(index)
        if ball_no>6:
            if len(list_extras)>0:
                match_data.loc[list_extras.pop(-1),'extras_type']='w'
    for i in list_extras:
        match_data.loc[i,'extras_type']='b'
    return match_data

In [742]:
def prepare_scorecard(match_data):
    match_data=get_extras_type(match_data)
#     print(match_data[match_data['bowler']==	'Mashrafe Mortaza'])
    players=list((match_data['striker'].append(match_data['non-striker']).append(match_data['bowler'])).unique())
    #     to make 22 players if any player has not played
#     for i in range(len(players),22):
#         players.append('p_'+str(i))
#         fow
#     player_stats=['match_no','bats_innings','name','position','over_batting','score','balls_bat','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no','fow','fow_runs','fow_overs','fow_batsman','fow_bowler']
    player_stats=['match_no','bats_innings','name','position','over_batting','score','balls_bat','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no']
    player_data={key:{key_type:0 for key_type in player_stats} for key in players}
    for p in players:
        player_data[p]['match_no']=match_data.loc[0,'file_no']
        player_data[p]['name']=p
    team_score=0
    balls=0
    pos=1
    inning=False
    w=1
    p_no=1
    w_no=1
    for index, row in match_data.iterrows():
        ov=str(row['over']).split('.')
        ball_no=int(ov[1])
        over_no=int(ov[0])
        if row['over']==0.1:
            pos=1
            w=1
            team_score=0
            w_no=1
            runs_over=0
            if row['innings']!=1:
                p_no=12
        if row['runs']==1:
            player_data[row['striker']]['ones']+=1
        elif row['runs']==2:
            player_data[row['striker']]['twos']+=1
        elif row['runs']==3:
            player_data[row['striker']]['threes']+=1
        elif row['runs']==4:
            player_data[row['striker']]['fours']+=1
        elif row['runs']==6:
            player_data[row['striker']]['sixes']+=1

        if player_data[row['striker']]['position']==0:
            player_data[row['striker']]['position']=pos
            player_data[row['striker']]['over_batting']=row['over']
            pos+=1
        if player_data[row['non-striker']]['position']==0:
            player_data[row['non-striker']]['position']=pos
            player_data[row['non-striker']]['over_batting']=row['over']
            pos+=1
    # wicket
#         print(row['out-player'])
        if not pd.isna(row['out-player']):
            player_data[row['out-player']]['out']=True
#             fow
#             player_data[players[p_no-1]]['fow']=w
#             player_data[players[p_no-1]]['fow_runs']=team_score
#             player_data[players[p_no-1]]['fow_overs']=row['over']
#             player_data[players[p_no-1]]['fow_batsman']=row['out-player']
#             player_data[players[p_no-1]]['fow_bowler']=row['bowler']
            p_no+=1
            w+=1
            if row['out']!='run out':
                player_data[row['bowler']]['wickets']+=1
            player_data[row['out-player']]['wicket_score']=team_score
            player_data[row['out-player']]['wicket_overs']=row['over']
            player_data[row['out-player']]['wicket_no']=w_no
            w_no+=1

        team_score+=row['runs']+row['extras']
        runs_over+=row['runs']
        if row['extras']!=0 and row['extras_type']=='w':
            runs_over+=1
            player_data[row['bowler']]['bowler_extras']+=1
            player_data[row['striker']]['score']+=row['extras']-1
            player_data[row['bowler']]['runs_bowled']+=row['extras']-1
        elif row['extras']!=0:
#             print(row)
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['striker']]['balls_bat']+=1
        else:
            player_data[row['striker']]['score']+=row['runs']
            player_data[row['striker']]['balls_bat']+=1
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['bowler']]['runs_bowled']+=row['runs']
        if ball_no>=6:
            if ball_no==6 and runs_over==0:
                player_data[row['bowler']]['maidens']+=1
            runs_over=0
        player_data[row['striker']]['bats_innings']=row['innings']
        if row['innings']==1:
            player_data[row['bowler']]['bats_innings']=2
        else:
            player_data[row['bowler']]['bats_innings']=1
#     print(player_data)
    player_data=[value for key, value in player_data.items()]
    scorecard=pd.DataFrame(data=player_data)
    scorecard=scorecard[player_stats]
    return scorecard

In [ ]:
i=0
for file in files[:]:
    print(i,'--',file)
    i+=1
    count=0
    df_index=[]
    df_row=[]
    add=r'./all_csv/'+file
    df_index=['file_no']
    file_no=file.split('.')[0]
    df_row=[file_no]
    with open(add) as f:
        new_f=f.readlines()
        for line in new_f:
            if 'version' in line:
                count+=1
            elif 'info' in line:
                count+=1
                line=line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index=rename_date_umpire(df_index)
                df_dic=dict(zip(df_index,df_row),index=[0])
                temp_info_df=pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                gender=df_info['gender'].iloc[0]
                gender=str.lower(gender.strip())
                break
    temp_df=pd.read_csv(add,skiprows=count,names=[0,'innings','over','batting-team','striker','non-striker','bowler','runs','extras','out','out-player'])
    temp_df=temp_df.drop([0],axis=1)
    temp_df['file_no']=[file_no]*(temp_df.shape[0])
    temp_sc=prepare_scorecard(temp_df)
#     append_file(temp_df,gender,type_game)
    append_file(temp_sc,temp_info_df,find_game(temp_sc,temp_info_df))

0 -- 1019975.csv
1 -- 682919.csv
2 -- 952191.csv


/home/sb/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


3 -- 1043961.csv
4 -- 565820.csv
5 -- 335990.csv
6 -- 980959.csv
7 -- 995463.csv
8 -- 1082599.csv
9 -- 1051297.csv
10 -- 562446.csv
11 -- 1144499.csv
12 -- 299008.csv
13 -- 524921.csv
14 -- 914233.csv
15 -- 946553.csv
16 -- 336006.csv
17 -- 980951.csv
18 -- 749797.csv
19 -- 1085970.csv
20 -- 682953.csv
21 -- 467887.csv
22 -- 534219.csv
23 -- 1089138.csv
24 -- 981003.csv
25 -- 247507.csv
26 -- 951347.csv
27 -- 533311.csv
28 -- 287869.csv
29 -- 743963.csv
30 -- 897747.csv
31 -- 736443.csv
32 -- 640935.csv
33 -- 476596.csv
34 -- 913621.csv
35 -- 1043953.csv
36 -- 534215.csv
37 -- 523733.csv
38 -- 257772.csv
39 -- 419110.csv
40 -- 501213.csv
41 -- 778023.csv
42 -- 756773.csv
43 -- 501212.csv
44 -- 538071.csv
45 -- 754739.csv
46 -- 533283.csv
47 -- 352663.csv
48 -- 1152557.csv
49 -- 527683.csv
50 -- 533301.csv
51 -- 1062574.csv
52 -- 293076.csv
53 -- 598051.csv
54 -- 249752.csv
55 -- 433574.csv
56 -- 401075.csv
57 -- 1073420.csv
58 -- 426423.csv
59 -- 951313.csv
60 -- 412696.csv
61 -- 87555

In [730]:
odi_info.columns

Index([], dtype='object')

In [731]:
ttwenty_info.columns

Index(['file_no', 'team', 'gender', 'season', 'date_0', 'competition',
       'match_number', 'venue', 'city', 'toss_winner', 'toss_decision',
       'player_of_match', 'umpire_0', 'umpire_1', 'reserve_umpire',
       'tv_umpire', 'match_referee', 'winner', 'winner_runs', 'index'],
      dtype='object')

In [732]:
odi_scorecard

""


In [733]:
ttwenty_info

,file_no,team,gender,season,date_0,competition,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire_0,umpire_1,reserve_umpire,tv_umpire,match_referee,winner,winner_runs,index
0,501264,Deccan Chargers,male,2011,2011/05/21,Indian Premier League,67,Himachal Pradesh Cricket Association Stadium,Dharamsala,Kings XI Punjab,field,S Dhawan,Asad Rauf,AM Saheba,K Bharatan,SL Shastri,GF Labrooy,Deccan Chargers,82,0


In [734]:
ttwenty_scorecard

,match_no,bats_innings,name,position,over_batting,score,balls_bat,ones,twos,threes,...,sixes,out,balls_bowl,maidens,runs_bowled,wickets,bowler_extras,wicket_score,wicket_overs,wicket_no
0,501264,1,S Dhawan,1,0.1,100,57,27,3,0,...,1,0,0,0,0,0,0,0,0.0,0
1,501264,1,DB Ravi Teja,2,0.1,60,41,16,3,2,...,2,True,0,0,0,0,0,131,13.2,1
2,501264,1,JP Duminy,3,13.3,12,10,6,0,0,...,1,True,24,0,23,1,0,158,16.1,2
3,501264,1,CL White,4,16.2,15,12,7,4,0,...,0,0,0,0,0,0,0,0,0.0,0
4,501264,2,PC Valthaty,1,0.1,5,6,1,0,0,...,0,True,24,0,23,1,1,9,1.4,1
5,501264,2,AC Gilchrist,2,0.1,51,37,5,5,0,...,2,True,0,0,0,0,0,83,10.4,3
6,501264,2,SE Marsh,3,1.5,13,8,1,0,0,...,0,True,0,0,0,0,0,22,2.6,2
7,501264,2,KD Karthik,4,3.1,11,14,7,0,0,...,0,True,0,0,0,0,0,84,10.6,4
8,501264,2,R McLaren,5,10.5,9,14,7,1,0,...,0,True,12,0,24,0,0,102,15.3,6
9,501264,2,AM Nayar,6,11.1,3,7,3,0,0,...,0,True,0,0,0,0,0,91,13.3,5
